In [1]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm

from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
# Obviously it is not a great idea to ignore warnings, however there are quite many
# RuntimeWarnings because of division by 0 in some parts of this notebook. To keep
# the instructive nature of this example notebook, these warnings are ignored.

In [5]:
pwd

'/scratch/usr/shkifmzb/notebooks/shared-notebooks/hiwi/old_std'

In [6]:
maskpath = '/scratch/usr/shkifmzb/ORION10/'
meshpath = [maskpath + '1_mesh_mask.nc']
mask_acc = xr.open_dataset('/scratch/usr/shkifmmp/master/data/ORION10/interpolated/int_mask_regions.nc').mask_regions
def data_for_det(exp, years):
    '''years =[year1, year2], exp = 5 or 6 '''
    datapath = '/scratch/usr/shklvn09/SCRATCH/ORION10.L46.LIM2vp.CFCSF6.MOPS.JRA.XIOS2.5.LP01-EXP0'+str(exp)+'/OUT/chunked_for_EDT/'
    data_in = [sorted(glob(datapath + '*'+str(year)+'0101_'+str(year)+'1231_grid_[TUV]_chunked_1klev.nc')) for year in range (years[0],years[-1])]
    return data_in

In [7]:
date = [2014, 2016]
data_in =data_for_det(5, date)
len(data_in)


2

In [8]:
date

[2014, 2016]

In [9]:
def inter(data_in, years):
    data=[]
# define metrics for xgcm (not strictly necessary)
    for i in range(0,len(data_in)):
        data1 = load_xorca_dataset(data_files=data_in[i], aux_files=meshpath, model_config='NEST',
               input_ds_chunks = {"time_counter": 1, "t": 1,
                              "z": 1, "deptht": 1, "depthu": 1, "depthv": 1, "depthw": 1,
                              "x": 1002, "y": 629},
               target_ds_chunks = {"t": 1,
                               "z_c": 1, "z_l": 1,
                               "x_c": 1002, "x_r": 1002, "y_c": 629, "y_r": 629})
        at, au = data1['e1t'] * data1['e2t'], data1['e1u'] * data1['e2u'] 
        av, af = data1['e1v'] * data1['e2v'], data1['e1f'] * data1['e2f'] 
        vt, vu, vv, vw = data1['e3t'] * at, data1['e3u'] * au, data1['e3v'] * av, data1['e3w'] * at 

        data1 = data1.update({'at': at, 'au': au, 'av': av, 'af': af, 'vt': vt, 'vu': vu, 'vv': vv, 'vw': vw})
        data1 = data1.set_coords(['at', 'au', 'av', 'af', 'vt', 'vu', 'vv', 'vw'])
        data.append(data1)
    metrics = {
        ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
        ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
#    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w'], # Z distances
        ('X', 'Y'): ['at', 'au', 'av', 'af'], # Areas
#    ('X', 'Y', 'Z'): ['vt', 'vu', 'vv', 'vw'] # Volumes
    }
    
    bathy = xr.open_mfdataset('../Bathymetry.nc')
    #mask_acc = xr.open_dataset('/scratch/usr/shkifmmp/master/data/ORION10/mask/mask_nest_int_2d.nc')
    #data = [data[i].update({'mask_acc': (['y_c', 'x_c'], mask_acc['mask_nest'].data)}) for i in range(0,len(data_in))]
    #ice= xr.open_mfdataset('../interpolated_Sea_Ice_1958_2018/Sea_Ice_on_fpoint_1959.nc')
    data = [data[i].update({'bathymetry': (['y_r', 'x_r'], bathy['Bathymetry'].data)}) for i in range(0,len(data_in))]
    #data = data.update({'ileadfra': (['t','y_r', 'x_r'], ice['ileadfra'].data)})
    grid = [xgcm.Grid(data[i], metrics=metrics) for i in range(0,len(data_in))]
    
   # Calculate vorticity and Okubo-Weiss parameter and make sure the chunk sizes are as before.

    data_OW = [et.okuboweiss.calc(data[i].isel(z_c=9, z_l=9), grid[i],
                             'vozocrtx', 'vomecrty').chunk({'x_c': 1002, 'x_r': 1002,
                                                            'y_c': 629, 'y_r': 629}) for i in range(0,len(data_in))]
    
    # Merge the new variables `OW` and `vort` to the dataset `data`
    data = [xr.merge([data[i], data_OW[i]], compat='override') for i in range(0,len(data_in))]
    
    interpolation_parameters = [{'model': 'ORCA',
                            'grid': 'latlon',
                            'start_time': str(years[0])+'-01-01', # time range start
                            'end_time': str(years[-1])+'-12-31', # time range end
                            'calendar': 'standard', # calendar, must be either 360_day or standard
                            'lon1': 77.5, # minimum longitude of detection region
                            'lon2': 69.5, # maximum longitude
                            'lat1': -67, # minimum latitude
                            'lat2': -30, # maximum latitude
                            'res': 1./10., # resolution of the fields in degrees
                            'vars_to_interpolate': ['OW', 'vort'], # variables to be interpolated 
                            'mask_to_interpolate': ['fmask', 'tmask', 'bathymetry'], # masks to interpolate
                            'regrid_method': 'bilinear', # method used for regridding (default is 'bilinear')
                            'ext_method': None} for year in range(years[0],years[-1])]
    
    data_int, regridder =[] , []

    for i in range(0,len(data)):

        data_int1, regridder1 = et.interp.horizontal(data[i], metrics, interpolation_parameters[i], weights=None, avoid_regrid=False)
        data_int.append(data_int1)
        regridder.append(regridder1)
    return data_int 

In [10]:
data_int = inter(data_in, date)

Interpolating from model grid: ORCA
No regridding necessary, just interpolating to vorticity grid point.
Interpolating OW
Interpolating vort
Interpolating fmask
Interpolating tmask
Interpolating bathymetry
Interpolating e1f
Interpolating e2f
Interpolating from model grid: ORCA
No regridding necessary, just interpolating to vorticity grid point.
Interpolating OW
Interpolating vort
Interpolating fmask
Interpolating tmask
Interpolating bathymetry
Interpolating e1f
Interpolating e2f


In [11]:
data_int=[data_int[i].compute() for i in range(0,len(data_in))]

In [12]:
#from scipy.signal import convolve

#def nanconv(a, k, MODE):
#    # Flat function for comparison.
#    o = np.ones(np.shape(a))
#   # Flat function with NaNs for comparison.
#   on = np.ones(np.shape(a))
#    # Find all the NaNs in the input and replace with 0
#    an = np.where(~np.isnan(a), a, 0)
#   on = np.where(~np.isnan(a), on, 0)
#    # Calculate what a 'flat' function looks like after convolution.
#    flat = convolve(on, k, mode=MODE)
#   #
#   # The line above will automatically include a correction for edge 
3   # effects,
#   # so remove that correction if the user does not want it.
#    flat = flat / convolve(o, k, mode=MODE)
#    #
#    # Do the actual convolution
#    output = convolve(an, k, mode=MODE) / flat
#    return np.where(~np.isnan(a), output, np.nan)

#def spatial_std_zoe(data, wx, wy):
#    window = np.ones((1, wy, wx)) / (wy * wx)
#    ext = np.zeros(( np.shape(data)[0], 
#                    np.shape(data)[1] + wy, np.shape(data)[2] + wx))
#    ext[:,  int(wy/2):-int(wy/2), int(wx/2):-int(wx/2)] = data
#    ext[:,  0:int(wy/2), :] = ext[:, int(wy):int(wy/2):-1, :]
#    ext[ :, -int(wy/2)::, :] = ext[ :, -int(wy/2):-int(wy):-1, :]
#    ext[ :, :, 0:int(wx/2)] = ext[ :, :, int(wx):int(wx/2):-1]
#    ext[ :, :, -int(wx/2)::] = ext[ :, :, -int(wx/2):-int(wx):-1]
#    std_tmp1 = np.abs(ext - nanconv(ext, window, "same")) ** 2
#    std_tmp2 = nanconv(std_tmp1, window, "same") ** 0.5
#    output =  xr.DataArray(std_tmp2[ :, int(wy/2):-int(wy/2), int(wx/2):-int(wx/2)], 
 #                          coords=data.coords, dims=data.dims).mean("time") 
#    return output

3

In [13]:
#mean_OW_spatial_std = []
#for i in range(0,len(data_in)):

#    OW_tmp = data_int[i]['OW'].isel(time=slice(0, 20)).compute()#

#    OW_tmp = OW_tmp.where(OW_tmp != 0)#

#    lon_tmp = OW_tmp['lon'].where(OW_tmp['lon'] > 0, other=OW_tmp['lon'] + 360.)
#    OW_tmp=OW_tmp.assign_coords({'lon': lon_tmp})
#    wx = 100 # rolling window size in x-direction
#    wy = 100 # rolling window size in y-direction
#    mean_OW_spatial_std.append(spatial_std_zoe(OW_tmp, wx, wy))

In [14]:
#one std for whole SO
mean_OW_spatial_std = []

for i in range(0,len(data_in)):

    OW_tmp = data_int[i]['OW'].compute()

    OW_tmp = OW_tmp.where(OW_tmp != 0)

    lon_tmp = OW_tmp['lon'].where(OW_tmp['lon'] > 0, other=OW_tmp['lon'] + 360.)
    OW_tmp=OW_tmp.assign_coords({'lon': lon_tmp})
    mean_OW_spatial_std.append(OW_tmp.std(('lon', 'lat'), skipna=True).mean('time'))

In [15]:
data_int = [data_int[i].update({'OW_std': mean_OW_spatial_std[i]}) for i in range(0,len(data_in))]

In [16]:
data_int

[<xarray.Dataset>
 Dimensions:     (time: 73, z: 46, lat: 628, lon: 3561)
 Coordinates:
   * time        (time) datetime64[ns] 2014-01-03T12:00:00 ... 2014-12-29T12:0...
   * z           (z) int64 1 2 3 4 5 6 7 8 9 10 ... 37 38 39 40 41 42 43 44 45 46
   * lat         (lat) float32 -67.96 -67.93 -67.89 ... -29.81 -29.72 -29.63
   * lon         (lon) float32 75.5 75.6 75.7 75.8 75.9 ... 71.2 71.3 71.4 71.5
 Data variables:
     OW          (time, lat, lon) float64 9.836e-12 8.819e-13 ... 0.0 0.0
     vort        (time, lat, lon) float64 -4.005e-07 -2.538e-07 ... 0.0 0.0
     fmask       (z, lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
     tmask       (z, lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
     bathymetry  (lat, lon) float64 467.2 453.9 453.9 453.9 ... nan nan nan nan
     e1f         (lat, lon) float64 4.172e+03 4.172e+03 ... 9.666e+03 9.666e+03
     e2f         (lat, lon) float64 4.172e+03 4.172e+03 ... 9.666e+03 9.666e+03
     OW_std      float64 1

In [17]:
# Specify parameters for eddy detection

date1= range(date[0], date[-1])
detection_parameters= {}
for i in range(0,len(date1)):

    detection_parameters[i] = {'model': 'ORCA',
                        'grid': 'latlon',
                        'start_time': str(date1[i])+'-01-01', # time range start
                        'end_time': str(date1[i])+'-12-31', # time range end
                        'calendar': 'standard', # calendar, must be either 360_day or standard
                        'lon1': 77.5, # minimum longitude of detection region
                        'lon2': 69.5, # maximum longitude
                        'lat1': -67, # minimum latitude
                        'lat2': -30, # maximum latitude
                        'min_dep': 1000, # minimum ocean depth where to look for eddies in m
                        'res': 1./10., # resolution of the fields in degree
#                        'OW_thr': data_int['OW_std'].values,s# .values it gives you the numpy array of the values (no informatin on dime and coordinate names
                         'OW_thr': data_int[i]['OW_std'].values, #with the 2D std one can remove .values
                        'OW_thr_name': 'OW_std', # Okubo-Weiss threshold for eddy detection
                        'OW_thr_factor': -0.3, # Okubo-Weiss parameter threshold
                        'Npix_min': 10, # minimum number of pixels (grid cells) to be considered as eddy
                        'Npix_max': 421, # maximum number of pixels (grid cells)
                        'no_long': False, # If True, elongated shapes will not be considered
                        'no_two': False # If True, eddies with two minima in the OW
                                        # parameter and a OW > OW_thr in between  will not
                                        # be considered
                       }

In [18]:
detection_parameters

{0: {'model': 'ORCA',
  'grid': 'latlon',
  'start_time': '2014-01-01',
  'end_time': '2014-12-31',
  'calendar': 'standard',
  'lon1': 77.5,
  'lon2': 69.5,
  'lat1': -67,
  'lat2': -30,
  'min_dep': 1000,
  'res': 0.1,
  'OW_thr': array(1.47853152e-10),
  'OW_thr_name': 'OW_std',
  'OW_thr_factor': -0.3,
  'Npix_min': 10,
  'Npix_max': 421,
  'no_long': False,
  'no_two': False},
 1: {'model': 'ORCA',
  'grid': 'latlon',
  'start_time': '2015-01-01',
  'end_time': '2015-12-31',
  'calendar': 'standard',
  'lon1': 77.5,
  'lon2': 69.5,
  'lat1': -67,
  'lat2': -30,
  'min_dep': 1000,
  'res': 0.1,
  'OW_thr': array(1.49952532e-10),
  'OW_thr_name': 'OW_std',
  'OW_thr_factor': -0.3,
  'Npix_min': 10,
  'Npix_max': 421,
  'no_long': False,
  'no_two': False}}

In [19]:
eddies={}
for i in range(0,len(data_int)):
    eddies[i] = et.detection.detect_OW(data_int[i].isel(z=9), detection_parameters[i], 'OW', 'vort', regrid_avoided=False, 
                                 use_bags=False, use_mp=False)

preparing data for eddy detection (masking and region extracting etc.)
detection at time step  1  of  73
detection at time step  9  of  73
detection at time step  17  of  73
detection at time step  25  of  73
detection at time step  33  of  73
detection at time step  42  of  73
detection at time step  50  of  73
detection at time step  58  of  73
detection at time step  66  of  73
preparing data for eddy detection (masking and region extracting etc.)
detection at time step  1  of  73
detection at time step  9  of  73
detection at time step  17  of  73
detection at time step  25  of  73
detection at time step  33  of  73
detection at time step  42  of  73
detection at time step  50  of  73
detection at time step  58  of  73
detection at time step  66  of  73


In [20]:
for j in range(0,len(data_int)):
    for i in np.arange(0, len(eddies[j])):
        datestring = str(eddies[j][i][0]['time'])[0:10]
        with open('./EXP05/onestd_'
          + str(datestring) + '_eddies_OW0.3_npmin10_npmax421.pickle', 'wb') as f:
            pickle.dump(eddies[j][i], f, pickle.HIGHEST_PROTOCOL)
        f.close()

In [92]:
eddies = et.detection.detect_OW(data_int_acc, detection_parameters[0], 'OW_acc', 'vort_acc', regrid_avoided=False, 
                                 use_bags=False, use_mp=False)

preparing data for eddy detection (masking and region extracting etc.)
detection at time step  1  of  73
detection at time step  9  of  73
detection at time step  17  of  73
detection at time step  25  of  73
detection at time step  33  of  73
detection at time step  42  of  73
detection at time step  50  of  73
detection at time step  58  of  73
detection at time step  66  of  73


In [94]:
#for j in range(0,len(data_int)):
for i in np.arange(0, len(eddies)):
    datestring = str(eddies[i][0]['time'])[0:10]
    with open('./ACC/detected/test_'
          + str(datestring) + '_eddies_OW0.3_npmin20_npmax200.pickle', 'wb') as f:
        pickle.dump(eddies[i], f, pickle.HIGHEST_PROTOCOL)
    f.close()